## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate source  \
0  2025-10-25-02-16-06 +0000    bbc   
1  2025-10-25-02-16-00 +0000    bbc   
2  2025-10-25-02-00-00 +0000    wsj   
3  2025-10-25-02-00-00 +0000    wsj   
4  2025-10-25-01-56-49 +0000    nyt   

                                               title  \
0  US beef prices are soaring. Will Trump's plans...   
1  Venezuela's Maduro says US 'fabricating war' a...   
2  Milei’s Overhaul of Argentina Has Another Prob...   
3      Why Bubbles Can Keep Inflating in Plain Sight   
4  Trump Administration Plans a Shake-Up at ICE t...   

                                                link  
0  https://www.bbc.com/news/articles/cr7m2pd3810o...  
1  https://www.bbc.com/news/articles/c891gzx7xn4o...  
2  https://www.wsj.com/world/americas/mileis-over...  
3  https://www.wsj.com/finance/stocks/why-bubbles...  
4  https://www.nytimes.com/2025/10/24/us/politics...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
4     trump     56
107  canada     16
108   trade     16
3      will     14
239    anti     14

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
349  2025-10-24-03-21-49 +0000     cbc   
80   2025-10-24-21-43-10 +0000  nypost   
350  2025-10-24-03-16-11 +0000  nypost   
125  2025-10-24-19-57-00 +0000     wsj   
138  2025-10-24-19-27-00 +0000     wsj   

                                                 title  \
349  Trump calls off trade talks with Canada over O...   
80   Ontario to pause anti-tariff Reagan ads after ...   
350  Trump terminates trade talks with Canada over ...   
125  Ontario to Pause Anti-Tariff Ad Campaign After...   
138  Ontario to Pause Anti-Tariff Commercial After ...   

                                                  link  score  
349  https://www.cbc.ca/news/world/trump-trade-nego...    162  
80   https://nypost.com/2025/10/24/us-news/ontario-...    159  
350  https://nypost.com/2025/10/23/us-news/trump-te...    159  
125  https://www.wsj.com/world/americas/ontario-to-...    158  
138  https://www.wsj.com/world/americas/ontario-to-...    138

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate   source  \
349    162  2025-10-24-03-21-49 +0000      cbc   
208     80  2025-10-24-15-22-00 +0000      wsj   
194     60  2025-10-24-16-12-41 +0000  latimes   
161     48  2025-10-24-17-59-23 +0000   nypost   
169     45  2025-10-24-17-43-51 +0000   nypost   
215     45  2025-10-24-14-59-00 +0000      wsj   
101     44  2025-10-24-20-56-50 +0000  latimes   
234     43  2025-10-24-13-54-28 +0000   nypost   
33      41  2025-10-24-23-42-29 +0000   nypost   
319     35  2025-10-24-07-34-38 +0000   nypost   

                                                 title  \
349  Trump calls off trade talks with Canada over O...   
208  Annual inflation heated up slightly in Septemb...   
194  U.S. sanctions Colombia's president, deploys a...   
161  Rubio warns West Bank annexation would threate...   
169  Influential House Minority Leader Hakeem Jeffr...   
215  The extended government shutdown has put colle...   
101  Video shows security opening fire on U-Haul tr...   
234  China, Russia use ‘asymmetric advantage,’ unle...   
33   Hunter Biden claims he didn’t think ‘laptop fr...   
319  Tropical Storm Melissa stationary in the Carib...   

                                                  link  
349  https://www.cbc.ca/news/world/trump-trade-nego...  
208  https://www.wsj.com/economy/consumer-price-ind...  
194  https://www.latimes.com/world-nation/story/202...  
161  https://nypost.com/2025/10/24/world-news/rubio...  
169  https://nypost.com/2025/10/24/us-news/hakeem-j...  
215  https://www.wsj.com/economy/shutdown-puts-octo...  
101  https://www.latimes.com/california/story/2025-...  
234  https://nypost.com/2025/10/24/business/china-r...  
33   https://nypost.com/2025/10/24/us-news/hunter-b...  
319  https://nypost.com/2025/10/24/world-news/tropi...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
